# Problem Formulation

### What is the input?





the input data representing Reddit Fake Post Detection (by Looking Only at the Title)

*   conisting of only one feature that represent the title of the posts 
*   this data doesn't contain any missing data but contain some noise in the text which need to be cleaned and preprocessed




### What is the output?


The output is the label that specify if the post in Reddit is fack or not


*   label contain the prediction of the probability (0-1, float)




### What data mining function is required?

As I understand from this part of the slide


```
Data Mining Functions
1. Generalization and Summarization
2. Association and Correlation
3. Classification & Prediction
4. Clustering
5. Outlier/Anomaly Analysis
6. Time and Ordering 
7. Structure and Network Analysis
```

The data mining in this problem requires Classification & Prediction After cleaning the data by:
* handeling any missing data
* removing duplications
* removing useless data (sach as text that contain very low number of characters)
* using the regular expression library (re)



### What could be the challenges?

The Challenges represented in:


*   Missing data
*   Noise data (sach as text that contain very low number of characters)
*   Dealing with text feature by preprocessing and preparing it before building the models
*   label that contain values out of 0 and 1








### What is the impact?

The impact of using the raw data as it is, without cleaning and reprocessing, will result a model with low accuracy that doesn't learn well or a desired from the data in the traing stage


---

The real-life impact of building a model that solve this problem is represent in limitting the number of fack posts that could be created and posted on Reddit site by detecting it and distinguish it from the real ones. only by looking at the Title.
<br/>

As the False information on the Internet has caused many social problems due to the raise of social network and its role in different domains such as politics.
<br/>
Solving this problem will lead to avoid the spreeding of these fack news 



### What is an ideal solution?

the ideal solution is to clean and preprocess the data before working with it

<br/>


*   Using word-level vectorizer instead of character-level vectorizer as it gives higher performance score while training 
*   Try different model with different hyperparamter search methods and different experimental protocols to obtain an ideal solution 
*   The best model that gives higher performance score was *LogisticRegression* followed by *XGBClassifier_Pipeline_RandomizedSearchCV_with_validation_set* 
 





### What is the experimental protocol used and how was it carried out? 

After loading the data and cleaning and preprocessing it, different experimental protocol are used through different trials:
* using cross_val_score to measure the performance while using any classifier with character-level vectorizer and word-level vectorizer
* using hyperparamter search method (RandomizedSearchCV) with validation set
* setting value for k-fold cross validation cv while using GridSearchCV / RandomizedSearchCV / or BayesSearchCV
<br/>

and measure the perormance using (roc_auc)


### What preprocessing steps are used?


*   view the data and understand it
*   using df.info() to get more insight about the data
*   check the missing data using df.isna().sum()
*   removing rows that contain lable = 2  
*   handeling the text feature by applying clean_text function in order to preprocesse and prepare it before building the models through:
  -   remove any html tags (< /br> often found)
  -   Keep only ASCII + European Chars and whitespace, no digits
  -   remove single letter chars
  -   convert all whitespaces (tabs etc.) to single wspace
  -   if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
  -   remove text that contain very low number of characters (consider it useless data)  
  - apply feature creation with TF-IDF (word vectorizer & character vectorizer)


# Get Started (Importing packages & Loading the data)


## Import packages 

In [ ]:
# this line is for BayesSearchCV and using skopt package
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 3.4 MB/s 


In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, KFold

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_score,recall_score, f1_score,precision_recall_curve
sns.set()
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.svm import SVC
from time import time
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import PredefinedSplit

import re
import pickle
import holoviews as hv
import nltk 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# some seeting for pandas and hvplot

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

## Load Data

In [ ]:
# Loading the data from csv files

train = pd.read_csv('data/xy_train.csv')
test = pd.read_csv('data/x_test.csv')

In [ ]:
# Look at first records of the data 
train.head()

,id,text,label
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0


In [ ]:
# drop the id column as it isn't a feature
train = train.drop(columns=['id'])

In [ ]:
# save the id of test data to use it in save the result
test_id = test['id']

# drop the id column as it isn't a feature
test = test.drop(columns=['id'])

In [ ]:
# show the information of the train dataset
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    60000 non-null  object
 1   label   60000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 937.6+ KB


# Data Cleaning and Preprocessing


In [ ]:
# check if there any missing values and counting them for each feature
train.isna().sum()

text     0
label    0
dtype: int64

In [ ]:
# Drop rows that contain lable = 2 
train = train[(train["label"] != 2)]
train

,text,label
0,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0
1,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0
2,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0
3,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0
4,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0
...,...,...
59995,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0
59996,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1
59997,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0
59998,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0


In [ ]:
# using different text preprocessing techniques
# the reference of this cell is 873_nlp.ipynb notebook

nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english"))


def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-z]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-z]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-z,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-z,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
%%time
# Clean text
train["text_clean"] = train.loc[train["text"].str.len() > 20, "text"]
train["text_clean"] = train["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)

CPU times: user 37 s, sys: 119 ms, total: 37.2 s
Wall time: 47.1 s


In [ ]:
test.isna().sum()

text    0
dtype: int64

In [ ]:
%%time
# Clean text
test["text_clean"] = test["text"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)

CPU times: user 13.4 s, sys: 91.3 ms, total: 13.5 s
Wall time: 13.5 s


In [ ]:
train.head()

,text,label,text_clean
0,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...


# Descriptive analysis


In [ ]:
from bokeh.models import NumeralTickFormatter
# Word Frequency of most common words
word_freq = pd.Series(" ".join(train["text_clean"]).split()).value_counts()
word_freq[1:40]

one         3285
like        3128
new         2998
look        2847
color       2737
man         2729
get         2602
trump       2578
say         2347
peopl       2316
use         2307
first       2248
make        2227
old         2226
time        2027
poster      2000
found       1999
day         1935
war         1858
post        1648
world       1570
work        1531
show        1513
us          1506
american    1504
take        1491
life        1482
psbattl     1470
help        1442
go          1420
state       1409
back        1369
two         1364
school      1345
see         1329
photo       1324
made        1314
right       1311
save        1308
dtype: int64

In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,angriff,1
1,delusion,1
2,wane,1
3,undament,1
4,miku,1
5,hatsun,1
6,nfler,1
7,hicock,1
8,mccall,1
9,wahr,1


In [ ]:
# Distribution of label
train["label"].value_counts(normalize=True)

0    0.538281
1    0.461719
Name: label, dtype: float64

# Feature creation with TF-IDF (word vectorizer)

In [ ]:
"""
Compute unique word vector with frequencies
exclude very uncommon (<10 obsv.) and common (>=30%) words
use pairs of two words (ngram)
"""
word_vectorizer = TfidfVectorizer(
    analyzer="word", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
)
word_vectorizer.fit(train["text_clean"])

TfidfVectorizer(max_df=0.3, min_df=10, ngram_range=(1, 2))

In [ ]:
word_vector = pd.Series(word_vectorizer.vocabulary_).sample(5, random_state=1)
print(f"Unique word (ngram) vector extract:\n\n {word_vector}")

Unique word (ngram) vector extract:

 eli       2666
go far    3595
bamboo     650
wisdom    9837
pocket    6705
dtype: int64


# Feature creation with TF-IDF (character vectorizer)

In [ ]:
"""
Compute unique word vector with frequencies
exclude very uncommon (<10 obsv.) and common (>=30%) words
use pairs of two words (ngram)
"""
char_vectorizer = TfidfVectorizer(
    analyzer="char", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
)
char_vectorizer.fit(train["text_clean"])

TfidfVectorizer(analyzer='char', max_df=0.3, min_df=10, ngram_range=(1, 2))

In [ ]:
char_vector = pd.Series(char_vectorizer.vocabulary_).sample(5, random_state=1)
print(f"Unique char (ngram) vector extract:\n\n {char_vector}")

Unique char (ngram) vector extract:

 xv    611
uv    529
bb     37
vn    548
ru    452
dtype: int64


# Spliting data


In [ ]:
# # Sample data - 25% of data to test set
# X_train, X_test, Y_train, Y_test = train_test_split(train["text_clean"], train["label"], random_state=1, test_size=0.25, shuffle=True, stratify=train["label"])

# print(X_train.shape)
# print(X_test.shape)

In [ ]:
# Y_train.value_counts(normalize=True)

In [ ]:
# split the train data to features and lable column
X_train = train["text_clean"]
Y_train = train["label"] 
print('original shape', X_train.shape, Y_train.shape)

original shape (59768,) (59768,)


# Saving Prediction Result


In [ ]:
# define function to save the csv file of the result after each trial
def saveResult(test, test_id, classifier, fileName):
  submission = pd.DataFrame()

  submission['id'] = test_id

  submission['label'] = classifier.predict_proba(test)[:,1]

  submission.to_csv(fileName, index=False)

# Modeling

## MLPClassifier with word-level vectorizer

thoughts and observations for trial 0, plan for trial 1: 

<br/>

I used **MLPClassifier** with **word-level vectorizer** function for transforming each sentence in our dataset to numeric vector to easely using it with our models then using the cross_val_score to measure the performance of the classifier.
<br/>

I excepected to get an acceptable and good performance score compare to using character-level vectorizer 



In [ ]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = word_vectorizer.transform(X_train)
test_vec = word_vectorizer.transform(test['text_clean'])
X_train_vec.get_shape()

(59768, 10061)

In [ ]:
%%time
clf = MLPClassifier(
        random_state=1,
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    )
scores = cross_val_score(clf, X_train_vec, Y_train, scoring="roc_auc", cv=5)
print(scores.mean())

0.8662701064700107
CPU times: user 31.7 s, sys: 30.9 s, total: 1min 2s
Wall time: 47.8 s


In [ ]:
%%time
clf.fit(X_train_vec, Y_train)

CPU times: user 8.1 s, sys: 6.82 s, total: 14.9 s
Wall time: 7.81 s


MLPClassifier(early_stopping=True, hidden_layer_sizes=(12, 12, 12),
              n_iter_no_change=1, random_state=1)

In [ ]:
saveResult(test_vec, test_id, clf, 'MLPClassifier_with_word_level_vectorizer.csv')

## MLPClassifier with character-level vectorizer

thoughts and observations for trial 1, plan for trial 2: 

<br/>

I used **MLPClassifier** with **character-level vectorizer** function for transforming each sentence in our dataset to numeric vector to easely using it with our models then using the cross_val_score to measure the performance of the classifier.
<br/>

I excepected to get lower performance score compare to using word-level vectorizer 



In [ ]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = char_vectorizer.transform(X_train)
test_vec = char_vectorizer.transform(test['text_clean'])
X_train_vec.get_shape()

(59768, 671)

In [ ]:
%%time
clf = MLPClassifier(
        random_state=1,
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    )
scores = cross_val_score(clf, X_train_vec, Y_train, scoring="roc_auc", cv=5)
print(scores.mean())

0.7120307231946029
CPU times: user 15.5 s, sys: 3.73 s, total: 19.2 s
Wall time: 15.6 s


In [ ]:
%%time
clf.fit(X_train_vec, Y_train)

CPU times: user 2.02 s, sys: 390 ms, total: 2.41 s
Wall time: 2 s


MLPClassifier(early_stopping=True, hidden_layer_sizes=(12, 12, 12),
              n_iter_no_change=1, random_state=1)

In [ ]:
saveResult(test_vec, test_id, clf, 'MLPClassifier_with_character_level_vectorizer.csv')

## RandomForestClassifier with word-level vectorizer

thoughts and observations for trial 2, plan for trial 3: 

<br/>

I used **RandomForestClassifier** with its defualt hyperparameter and with **word-level vectorizer** function for transforming each sentence in our dataset to numeric vector to easely using it with our models then using the cross_val_score to measure the performance of the classifier.
<br/>

I excepected to get an acceptable and good performance score 



In [ ]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = word_vectorizer.transform(X_train)
test_vec = word_vectorizer.transform(test['text_clean'])
X_train_vec.get_shape()

(59768, 10061)

In [ ]:
%%time
clf = RandomForestClassifier(random_state=1)
scores = cross_val_score(clf, X_train_vec, Y_train, scoring="roc_auc", cv=5)
print(scores.mean())

0.8366559553918183
CPU times: user 7min 31s, sys: 1.25 s, total: 7min 32s
Wall time: 7min 35s


In [ ]:
%%time
clf.fit(X_train_vec, Y_train)

In [ ]:
saveResult(test_vec, test_id, clf, 'RandomForestClassifier_with_word_level_vectorizer.csv')

## LogisticRegression with word-level vectorizer

thoughts and observations for trial 3, plan for trial 4: 

<br/>

I used **LogisticRegression** with its defualt hyperparameter and with **word-level vectorizer** function for transforming each sentence in our dataset to numeric vector to easely using it with our models then using the cross_val_score to measure the performance of the classifier.
<br/>

I excepected to get low performance score but it accually gives me the best score across all trials 



In [ ]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = word_vectorizer.transform(X_train)
test_vec = word_vectorizer.transform(test['text_clean'])
X_train_vec.get_shape()

(59768, 21026)

In [ ]:
%%time
clf = LogisticRegression(solver="sag", random_state=1)
scores = cross_val_score(clf, X_train_vec, Y_train, scoring="roc_auc", cv=10)
print(scores.mean())

0.8722398657969277
CPU times: user 7.89 s, sys: 29.7 ms, total: 7.92 s
Wall time: 7.51 s


In [ ]:
%%time
clf.fit(X_train_vec, Y_train)

CPU times: user 919 ms, sys: 2.21 ms, total: 921 ms
Wall time: 920 ms


LogisticRegression(random_state=1, solver='sag')

In [ ]:
saveResult(test_vec, test_id, clf, 'LogisticRegression_with_word_level_vectorizer.csv')

## SVC with word-level vectorizer

thoughts and observations for trial 4, plan for trial 5: 

<br/>

I used **SVM Classifier** with its defualt hyperparameter and with **word-level vectorizer** function for transforming each sentence in our dataset to numeric vector to easely using it with our models then using the cross_val_score to measure the performance of the classifier.
<br/>

I excepected to get higher performance score but in slow time, acually it gives me a reasonable score not the highest one and takes a lot of time more than i expected



In [ ]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = word_vectorizer.transform(X_train)
test_vec = word_vectorizer.transform(test['text_clean'])
X_train_vec.get_shape()

(59768, 10061)

In [ ]:
%%time
clf = SVC(probability=True)
scores = cross_val_score(clf, X_train_vec, Y_train, scoring="roc_auc", cv=3)
print(scores.mean())

0.8635064007438583
CPU times: user 3h 3min 17s, sys: 15.1 s, total: 3h 3min 32s
Wall time: 3h 3min 43s


In [ ]:
%%time
clf.fit(X_train_vec, Y_train)

CPU times: user 2h 20min 16s, sys: 10.2 s, total: 2h 20min 26s
Wall time: 2h 20min 58s


SVC(probability=True)

In [ ]:
saveResult(test_vec, test_id, clf, 'SVC_with_word_level_vectorizer.csv')

## XGBClassifier Pipeline (RandomizedSearchCV) with validation set

thoughts and observations for trial 5, plan for trial 6: 

<br/>

I used **XGBClassifier** through the pipeline that contain the classifier and the TfidfVectorizer which used to transform each sentence of the text data to numeric vector
with **RandomizedSearchCV** function to get the best random hyperparameters of all the hyperparameter combinations among the specified number of iteration
and with **Validation Set** 
<br/>

the hyperparameters that used in this trial:
* 'tfidf__ngram_range': [(1, 2), (1, 3)]
* 'tfidf__max_df': np.arange(0.3, 0.8)
* 'tfidf__min_df': np.arange(5, 100)
* 'n_estimators': [100, 150, 200]  
* 'max_depth':[200, 300, 500]  

<br/>

I excepected to get an acceptable and good performance score
<br/>
Also I excepected the hyperparameters that reach the *local optimal* (within the given range) and produce the better accuracy among the random selected combinations



In [ ]:
pipe = Pipeline([
                 ("tfidf", TfidfVectorizer()),
                 ('my_classifier', XGBClassifier())
                 ])

params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__n_estimators': [100, 150, 200],  
    'my_classifier__max_depth':[200, 300, 500]
}

In [ ]:
%%time
from sklearn.model_selection import PredefinedSplit

# Split the original training set to a train and a validation set
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, Y_train, train_size = 0.8, stratify = Y_train, shuffle=True, random_state = 2022)

# Create a list where train data indices are -1 and validation data indices are 0
# X_tr (new training set), X_train
split_index = [-1 if x in X_tr.index else 0 for x in X_train.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

pipe_clf = RandomizedSearchCV(
    pipe, params, cv=pds, verbose=1, n_jobs=2,
    # number of random trials
    n_iter=5,
    scoring='roc_auc')

# here we still use X_train; but the grid search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set
pipe_clf.fit(X_train, Y_train)

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 1 folds for each of 5 candidates, totalling 5 fits
best score 0.8482701068133212
best score {'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 9, 'tfidf__max_df': 0.3, 'my_classifier__n_estimators': 200, 'my_classifier__max_depth': 300}
CPU times: user 31min, sys: 5.15 s, total: 31min 5s
Wall time: 1h 42min 3s


In [ ]:
saveResult(test['text_clean'], test_id, pipe_clf, 'XGBClassifier_Pipeline_RandomizedSearchCV_with_validation_set.csv')

## Parameter tuning with XGBClassifier Pipeline (RandomizedSearchCV)

thoughts and observations for trial 6, plan for trial 7: 

<br/>

I used **XGBClassifier** through the pipeline that contain the classifier and the TfidfVectorizer which used to transform each sentence of the text data to numeric vector
with **RandomizedSearchCV** function to get the best random hyperparameters of all the hyperparameter combinations among the specified number of iteration
and with **cross-validation** 
<br/>

the hyperparameters that used in this trial:
* 'tfidf__ngram_range': [(1, 2), (1, 3)]
* 'tfidf__max_df': np.arange(0.3, 0.8)
* 'tfidf__min_df': np.arange(5, 100)
* 'n_estimators': [100, 150, 200]  
* 'max_depth':[100, 200, 300]  

<br/>

I excepected to get lower accuracy compared to the same classifier with the predefined validation set
<br/>
Also I excepected the hyperparameters that reach the *local optimal* (within the given range) and produce the better accuracy among the random selected combinations



In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ('my_classifier', XGBClassifier())])

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__n_estimators': [50, 100, 200],  
    'my_classifier__max_depth':[100, 200, 300]
}


pipe_clf = RandomizedSearchCV(
    pipe, params, cv=4, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=5,
    scoring='roc_auc')

pipe_clf.fit(X_train, Y_train)
pickle.dump(pipe_clf, open("./clf_pipe.pck", "wb"))

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 4 folds for each of 5 candidates, totalling 20 fits
best score 0.8357986772864227
best score {'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 18, 'tfidf__max_df': 0.3, 'my_classifier__n_estimators': 100, 'my_classifier__max_depth': 100}
CPU times: user 5min 2s, sys: 3.58 s, total: 5min 5s
Wall time: 1h 41min 10s


In [ ]:
saveResult(test['text_clean'], test_id, pipe_clf, 'XGBClassifier_Pipeline_with_RandomizedSearchCV.csv')

## Parameter tuning with XGBClassifier Pipeline (BayesSearchCV)

thoughts and observations for trial 7, plan for trial 8: 

<br/>

I used **XGBClassifier** through the pipeline that contain the classifier and the TfidfVectorizer which used to transform each sentence of the text data to numeric vector
with **BayesSearchCV** function to get the best hyperparameters among the specified number of iteration based on using bayesian learning to predict what is the next hyperparamter values we should try given the current trials
<br/>

the hyperparameters that used in this trial:
* 'tfidf__max_df': np.arange(0.3, 0.8)
* 'tfidf__min_df': np.arange(5, 100)
* 'n_estimators': [100, 200]  
* 'max_depth':[200, 300]   

<br/>

I excepected to get better accuracy compared to the RandomizedSearchCV with the same classifier
<br/>
Also I excepected the hyperparameters that reach the *local optimal* (within the given range) and produce the better accuracy among the generated combinations



In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", ngram_range=(1, 3), norm="l2")), ('my_classifier', XGBClassifier())])

# define parameter space to test # runtime 35min
params = {
    # "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__n_estimators': [100, 200],  
    'my_classifier__max_depth':[200, 300]
}


pipe_clf = BayesSearchCV(
    pipe, params, cv=3, verbose=1, n_jobs=2, n_iter=5,
    scoring='roc_auc')

pipe_clf.fit(X_train, Y_train)
#pickle.dump(pipe_clf, open("./clf_pipe.pck", "wb"))

print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score 0.8339057171638196
best score OrderedDict([('my_classifier__max_depth', 218), ('my_classifier__n_estimators', 173), ('tfidf__max_df', 0.3), ('tfidf__min_df', 19)])
CPU times: user 18min 16s, sys: 7.15 s, total: 18min 23s
Wall time: 2h 16min 11s


In [ ]:
saveResult(test['text_clean'], test_id, pipe_clf, 'XGBClassifier_Pipeline_with_GridSearchCV.csv')


## Parameter tuning with SVC Pipeline (RandomizedSearchCV)

thoughts and observations for trial 8, plan for trial 9: 

<br/>

I used **SVM Classifier** through the pipeline that contain the classifier and the TfidfVectorizer which used to transform each sentence of the text data to numeric vector
with **RandomizedSearchCV** function to get the best random hyperparameters of all the hyperparameter combinations among the specified number of iteration 
<br/>

the hyperparameters that used in this trial:
* 'tfidf__ngram_range': [(1, 2), (1, 3)]
* 'tfidf__max_df': np.arange(0.3, 0.8)
* 'tfidf__min_df': np.arange(5, 100)
* 'C': np.arange(0.2, 1, 0.15)

<br/>

I excepected better performance score compared to the SVC with its defualt hyperparameter and with word-level vectorizer 
<br/>

I excepected the hyperparameters that reach the *local optimal* (within the given range) and produce the better accuracy among the random selected combinations



In [ ]:
%%time

# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("my_classifier", SVC(probability=True))])

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    "my_classifier__C": np.arange(0.2, 1, 0.15),
}
# it is quite slow so we do 4 for now
pipe_clf = RandomizedSearchCV(
    pipe, params, n_jobs=-1, scoring="roc_auc", n_iter=3)
pipe_clf.fit(X_train, Y_train)


print('best score {}'.format(pipe_clf.best_score_))
print('best score {}'.format(pipe_clf.best_params_))

best score 0.8667282140889441
best score {'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 15, 'tfidf__max_df': 0.3, 'my_classifier__C': 0.9499999999999997}
Wall time: 2d 7h 26min 35s


In [ ]:
saveResult(test['text_clean'], test_id, pipe_clf, 'SVC_Pipeline_with_RandomizedSearchCV.csv')

# Questions

## What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?



Word n-gram model
* Represent **unique word sequence** of length n as
feature
Character n-gram model
* Represent **unique character sequence** of length n as
feature
<br/>

For a given document in a language, there are more character N-grams repetitions than word n-gram repetitions,
i.e. a character N-grams split highlights some common properties that a word n-gram split does not.



---

Word n-gram tends to suffer more from the OOV issue as:

**Out Of Vocabulary (OOV) words** refer to the new words which are encountered at testing. These new words do not exist in the vocabulary. Hence, these methods fail in handling OOV words.

<br/>


resources: 
* https://reader.elsevier.com/reader/sd/pii/S1877042813041918?token=3E1F8BE1DC047E8C3B89A98E9A583DE265B6477EF7E4DC237DACFA7F133D2A53A3592532CD63666A37288E8E65F64116&originRegion=eu-west-1&originCreation=20220314111053



## What is the difference between stop word removal and stemming? Are these techniques language-dependent?

Stemming
* Stemmers remove morphological affixes from words,
leaving only the word stem. (nltk)

ex:
* print(stemmer.stem("running"))
  - run
* print(stemmer.stem("generously"))
  - generous

<br/>

Stop words removal
* Remove high frequency words that have little semantic weight and are thus unlikely to help the retrieval process

ex:
* i, me, my, myself, we, our, ours, ourselves, you, your, yours,
yourself, yourselves, he, him, his, himself, she, her, hers,
herself, it, its, itself, they, them, their, theirs, …

<br/>

---

<br/>

Yes, both of these techniques are language-dependent. So we need to specify the language when using them
<br/>
example:

```
stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))
```




## Is tokenization techniques language dependent? Why?

No, not all tokenization techniques are language dependent


There are various tokenization techniques available which can be applicable based on the language and purpose of modeling.
<br/>

So, Tokenization is depending upon delimiters


<br/>

The basic initial step is converting texts using Tokenization method, which ‘breaks’ raw text in smaller pieces. Tokens can be words, singles characters or subwords (n-gram characters: a contiguous sequence of n items from a given sample of text or speech). 
<br/>

**The most common way of tokenization is based on space**. Taking space as a delimiter for example, you will get from “Natural Language Processing” 3 tokens : “Natural”, “Language”, “Processing”.

<br/>

Major techniques for tokenizing are:

Split, Spacy, Gensim, Regular Expression, NLTK (Natural Language Toolkit) library, Dictionary Based Tokenization, Rule Based Tokenization, Penn TreeBank Tokenization, and many others...

<br/>

Which Tokenization method to use will depend on your text (language, presence of white spaces or characters, special signs, type of genre (medical, scientific, etc.)) and tasks you need to achieve

<br/>

resources:
* https://www.linkedin.com/pulse/what-tokenization-natural-language-processing-nlp-rakesh-bhol
* https://medium.com/unpackai/natural-language-processing-tokenization-and-numericalization-63c2df20d917 

## What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?

TfidfVectorizer and CountVectorizer both are methods for converting text data into vectors as model can process only numerical data.

In CountVectorizer we only count the number of times a word appears in the document which results in biasing in favour of most frequent words. this ends up in ignoring rare words which could have helped is in processing our data more efficiently.

In TfidfVectorizer we consider overall document weightage of a word. It helps us in dealing with most frequent words. Using it we can penalize them. TfidfVectorizer weights the word counts by a measure of how often they appear in the documents.

<br/>

> TF-IDF is better than Count Vectorizers because it not only focuses on the frequency of words present in the corpus but also provides the importance of the words. We can then remove the words that are less important for analysis, hence making the model building less complex by reducing the input dimensions.

---

<br/>

No, it Wouldn't be feasible to use all possible n-grams especially if it is of a significant a size


As it depends on the application and the dataset. In some applications, it is important to deal with some combinations of words together as they are together, have a big effect. 

For problems like Sentiment Analysis, setting n-gram ranges that use bigrams or trigrams can dramatically improve the accuracy of classification, as they can capture more complex expressions formed by the composition of more than one word.

Another example: if we have a dataset that contain some products and we interested in the product that is easy to use, then the sentence 'easy to use' is important for us, therefore we need to use the trigrams (n-gram size = 3) to capture this combination of words, so this will lead to improve the accuracy of the model.

Also if we don't have a specific combination of word that is important to us, we can then experiment different value of n-grams whether doing it manually (trial and error) or by using one of hyperparamter search methods (grid/random/bayesian)

<br/>

resources:
* https://www.quora.com/What-is-the-difference-between-TfidfVectorizer-and-CountVectorizer-1
* https://help.monkeylearn.com/en/articles/2174105-n-gram-range